In [1]:
import numpy as np
import functools
import pandas as pd
import os
import copy
import operator

In [3]:
df = pd.read_csv('./In_situMeasurementforTraining_201712_1.csv')
df['hour'] = df['hour']*60-180
df_city = pd.read_csv('./CityData.csv')

In [4]:
def find_neighbors(x, limit_x=[1, 548], limit_y=[1, 421]):
    xx = [(x[0]+i_i, x[1]+i_j) for i_i in [-1, 0, 1] for i_j in [-1, 0, 1] if abs(i_i)+abs(i_j)==1]
    
    xx = list(filter(lambda i: i[0]>=limit_x[0] and i[0]<=limit_x[1] and i[1]>=limit_y[0] and i[1]<=limit_y[1], xx))
    return xx

In [5]:
def pointwise_compare(x):
    x2 = np.concatenate([[np.nan], x[:-1]])
    return x!=x2

In [6]:
def _reconcile(s1, s2):
    pts = np.unique(np.sort(np.concatenate([s1._x, s2._x])))
    # Handle case when endpoints are inf
    cpts = pts.copy()
    cpts[0] = min(np.min(cpts[1:]), 0.) - 1
    cpts[-1] = max(np.max(cpts[:-1]), 0.) + 1
    mps = (cpts[1:] + cpts[:-1]) / 2.
    return [(pts, s(mps)) for s in (s1, s2)]


def _same_support(s1, s2):
    return np.all(s1._x[[0, -1]] == s2._x[[0, -1]])


def require_compatible(f):
    @functools.wraps(f)
    def wrapper(self, other, *args, **kwargs):
        if isinstance(other, StepFunction) and not _same_support(self, other):
            raise TypeError("Step functions have different support: %s vs. %s" % (
                self._x[[0, -1]], other._x[[0, -1]]))
        return f(self, other, *args, **kwargs)
    return wrapper


class StepFunction:
    '''A step function.'''

    def __init__(self, x, y):
        '''Initialize step function with breakpoints x and function values y.
        x and y are arrays such that
            f(z) = y[k], x[k] <= z < x[k + 1], 0 <= k < K.
        Thus, len(x) == len(y) + 1 and domain of f is (x[0], x[K]).
        '''
        if len(x) != 1 + len(y):
            raise RuntimeError("len(x) != 1 + len(y)")
        self._x = np.array(x)
        self._y = np.array(y, dtype='float')
        self._compress()

    @property
    def K(self):
        '''The number of steps.'''
        return len(self._y)

    def _compress(self):
        # Combine steps which have equal values
     #   ny = np.concatenate([[np.nan], self._y, [np.nan]])
     #   ys = np.diff(ny) != 0
        ys = np.concatenate([pointwise_compare(self._y), [True]])
        self._x = self._x[ys]
        self._y = self._y[ys[:-1]]

    def _binary_op(self, other, op, desc):
        if isinstance(other, StepFunction):
            (s1_x, s1_y), (s2_x, s2_y) = _reconcile(self, other)
            return StepFunction(s1_x, op(s1_y, s2_y))
        # Fall back to normal semantics otherwise
        return StepFunction(self._x, op(self._y, other))

    def __add__(self, other):
        return self._binary_op(other, operator.add, "add")

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        return self._binary_op(other, operator.sub, "subtract")

    def __rsub__(self, other):
        return -self + other

    def __mul__(self, other):
        return self._binary_op(other, operator.mul, "multiply")

    def __rmul__(self, other):
        return self * other

    def __div__(self, other):
        return self._binary_op(other, operator.div, "divide")

    def __rdiv__(self, other):
        return (self ** -1) * other

    # Unary operations

    def __neg__(self):
        return StepFunction(self._x, -self._y)

    def __pow__(self, p):
        return StepFunction(self._x, pow(self._y, p))

    def __abs__(self):
        return StepFunction(self._x, abs(self._y))

    # Equality and comparison operators

    @require_compatible
    def __eq__(self, other):
        return (np.array_equal(self._x, other._x) and 
                np.array_equal(self._y, other._y))

    @require_compatible
    def __lt__(self, other):
        diff = other - self
        return np.all(diff._y > 0)

    @require_compatible
    def __le__(self, other):
        diff = other - self
        return np.all(diff._y >= 0)

    @require_compatible
    def __gt__(self, other):
        return -self < -other

    @require_compatible
    def __ge__(self, other):
        return -self <= -other

    def __call__(self, s):
        return self._y[np.searchsorted(self._x, s, side="right") - 1]

    def __repr__(self):
        return "StepFunction(x=%s, y=%s)" % (repr(self._x), repr(self._y))

    def integral(self):
        nz = self._y != 0
        d = np.diff(self._x)
        return (d[nz] * self._y[nz]).sum()
    
    def find_interval_value(self, x_interval):
        # [a, b)
     
        y = np.arange(np.argwhere(self._x<=x_interval[0])[-1], np.argwhere(self._x<x_interval[1])[-1]+1)
        yy = self._y[y]
        yyy = np.sort(np.unique(np.append(self._x[y], x_interval)))
        yyy = yyy[np.all(np.array([yyy>=x_interval[0], yyy<=x_interval[1]]), axis=0)]
        return np.array([yyy, yy])
   
    def extend_domain(self, new_value):
 
        if new_value < self._x[0]:
            self._x = np.insert(self._x, 0, new_value)
            self._y = np.insert(self._y, 0, float("Inf"))
            self.__init__(self._x, self._y)
        elif new_value > self._x[0]:
            self._x = np.append(self._x, new_value)
            self._y = np.append(self._y, float("Inf"))
            #self.__init__(self._x, self._y)
        else:
            pass
            

In [7]:
def hvdistance(x, y):
    return np.sum(np.abs(np.array(x) - np.array(y)))

In [34]:
class vertex_i:
    def __init__(self, vi, vs, ve, td, ta):
        self.vi = vi
        self.Ti = np.array([hvdistance(vi, vs)*2, ta])
        self.Si = np.array([ta, ta])
        self.TAUi = None
        self.negihbor_Sij = {}
        self.ta = ta
        self.td = td
        self.ve = ve
    #def update_Si(self):
        
    def update_TAUi_Si(self, interval_x):
        f = StepFunction(self.gi_d, self.gi_v)
        y = f.find_interval_value(interval_x)
        self.TAUi = y[1].min()
        self.TAUi2 = y[1].min() + hvdistance(self.vi, self.ve)
        # self.Si = np.array([y[0][np.argmin(y[1])], self.ta])

        self.Si = np.array([min(self.gi_d[np.append(self.gi_v == self.TAUi, [False])]), self.ta])
    
    
    def create_negihbor(self, ta):
        
        x = find_neighbors(self.vi, limit_x=[1, 548], limit_y=[1, 421])
        y = [[ta-2, ta-2]]*len(x)
        
        self.negihbor_Sij = dict(zip(x, y))
    
    def create_gi(self):
        self.gi_d = np.array([self.td, self.ta])
        self.gi_v = np.array([float('Inf')])
    
    def update_gi(self, i_x, i_y):
          
        i_x = np.array(i_x)
        i_y = np.array(i_y)
       
        f1 = StepFunction(self.gi_d, self.gi_v)
        
        f2 = StepFunction(i_x, i_y)
        
        if 0 not in i_x:
            f2.extend_domain(self.td)
        if 1080 not in i_x:
            f2.extend_domain(self.ta)
        
        x1 = np.sort(np.unique(np.append(self.gi_d, i_x)))
        y1 = [min(f1(i), f2(i)) if i in i_x else f1(i) for i in x1[:-1]]
        
        f = StepFunction(x1, y1)
        
        self.gi_d = f._x
        self.gi_v = f._y
    
        

In [35]:
def compute_minimum_cost(v_s, v_e, t_d, t_a, df):

    Q = {}
    QQ = []
    Q[v_s] = vertex_i(v_s, v_s, v_e, t_d, t_a)
    Q[v_s].gi_v = np.array([0])
    Q[v_s].gi_d = np.array([t_d, t_a])
    t = Q[v_s].Ti
    Q[v_s].update_TAUi_Si([t_d, t_a])
    Q[v_s].create_negihbor(t_a)
    
    
    v_i = v_s
    run_times = 1
    while v_i != v_e:
        
        for i in find_neighbors(v_i, limit_x=[1, 548], limit_y=[1, 421]):
            # if (Q[v_i].Si[0] < (t_a - 2)) and (Q[v_i].TAUi != float('Inf')):
            if (Q[v_i].Si[0] < (t_a - 2)):
                R_ij = [Q[v_i].Si[0], t_a-2]
                 
               
                x = [R_ij[0], Q[v_i].negihbor_Sij[i][0]]
                
                
                domain_ij = [x[0]+2, x[1]+2]
        
            
              
                if domain_ij[0] < domain_ij[1]:
                    f_ij = df[(df['xid'] == i[0]) & (df['yid'] == i[1])]
                    f_ij.sort_values(by='hour', ascending=True, inplace=True)
                
              
                    f_ij = StepFunction(np.append(f_ij.hour, t_a), np.where(f_ij.wind>=15, 1440, 0))
                    
                    
                    f_ij = f_ij.find_interval_value(domain_ij)
                
                    update_d = f_ij[0]
                    update_v = f_ij[1] + Q[v_i].TAUi
                
                    Q[v_i].negihbor_Sij[i] = R_ij
                
                    if i not in [iii for iii in Q]:
                        Q[i] = vertex_i(i, v_s, v_e, t_d, t_a)
                        Q[i].create_gi()
                        Q[i].create_negihbor(t_a)
                    
                    domain_j = [Q[i].Ti[0], Q[i].Si[0]]
                    
                
                    
                    if Q[i].Ti[0] < Q[i].Si[0]:
                        Q[i].update_gi(update_d, update_v)

                
                        run_times += 1
                        #print run_times
                        #print 'update %s in %s runs' % (i, run_times)
                
                        Q[i].update_TAUi_Si(domain_j)
                
 
                    if i not in [iii for iii, jjj in QQ]:
                        QQ.append((i, Q[i].TAUi2))
                              
                
                
        if (Q[v_i].Ti[0] < Q[v_i].Si[0]):
            
            domain_i = [Q[v_i].Ti[0], Q[v_i].Si[0]]
            
            Q[v_i].update_TAUi_Si(domain_i)
            #print domain_i
            #print Q[v_i].gi_d
            #print Q[v_i].gi_v
            #print '----------'
            
            QQ.append((v_i, Q[v_i].TAUi2))
            
        
        QQ = sorted(QQ, key=operator.itemgetter(1), reverse=False)
        
        v_i = QQ.pop(0)[0]
    return Q

In [85]:
def path_selection(g, ti, v_s, v_e, df, path, arrival_time, break_search):
    
    
    v_i = v_e
    
    path.append(v_i)
    arrival_time.append(ti)
    
    if v_i == v_s or break_search:
        return {'path': path, 'arrival_time': arrival_time}
    else:
        break_search = True
        stop_find = False
        gi = StepFunction(g[v_i].gi_d, g[v_i].gi_v) 
        
        f_ji = df[(df['xid'] == v_i[0]) & (df['yid'] == v_i[1])]
        f_ji.sort_values(by='hour', ascending=True, inplace=True)        
        f_ji = StepFunction(np.append(f_ji.hour, g[v_i].ta), np.where(f_ji.wind>=15, 1440, 0))
        
        for j in find_neighbors(v_i, limit_x=[1, 548], limit_y=[1, 421]):
            
            if stop_find:
                continue
        
            if j in g:
                
                tj = g[j].gi_d[np.argmin(g[j].gi_v)]
                gj = StepFunction(g[j].gi_d, g[j].gi_v)

                
                find_idx_j = (g[j].gi_d <= (ti-2)) & np.append((g[j].gi_v == (gi(ti) - f_ji(ti))), [False])

                if sum(find_idx_j) > 0:
                    
                #    print g[j].gi_d[g[j].gi_d <= (ti-2)]
                #    print g[j].gi_v[g[j].gi_v == (gi(ti) - f_ji(ti-2))]
                #    print g[j].gi_d[find_idx_j]
                #    print np.max(g[j].gi_d[find_idx_j])
                #    print '----------'
               
                    tj = np.max(g[j].gi_d[find_idx_j])
                    
                    #arrival_time.append(ti-2-tj)
                    
                    v_i = j
                    ti = tj
                    
                    stop_find = True
                    break_search = False
        return path_selection(g, ti, v_s, v_i, df, path, arrival_time, break_search)
        
            

In [86]:
df_city

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234
4,4,236,241
5,5,315,281
6,6,358,207
7,7,363,237
8,8,423,266
9,9,125,375


In [78]:
p_result = compute_minimum_cost((142, 328), (423, 266), 0, 1080, df)

/home/k1758472/.conda/envs/DS_Py27/lib/python2.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2
update (141, 328) in 2 runs
3
update (142, 327) in 3 runs
4
update (142, 329) in 4 runs
5
update (143, 328) in 5 runs
6
update (141, 327) in 6 runs
7
update (142, 326) in 7 runs
8
update (143, 327) in 8 runs
9
update (141, 326) in 9 runs
10
update (142, 325) in 10 runs
11
update (143, 326) in 11 runs
12
update (141, 325) in 12 runs
13
update (142, 324) in 13 runs
14
update (143, 325) in 14 runs
15
update (141, 324) in 15 runs
16
update (142, 323) in 16 runs
17
update (143, 324) in 17 runs
18
update (141, 323) in 18 runs
19
update (142, 322) in 19 runs
20
update (143, 323) in 20 runs
21
update (141, 322) in 21 runs
22
update (142, 321) in 22 runs
23
update (143, 322) in 23 runs
24
update (141, 321) in 24 runs
25
update (142, 320) in 25 runs
26
update (143, 321) in 26 runs
27
update (141, 320) in 27 runs
28
update (142, 319) in 28 runs
29
update (143, 320) in 29 runs
30
update (141, 319) in 30 runs
31
update (142, 318) in 31 runs
32
update (143, 319) in 32 runs
33
update (141, 318) in 

254
update (164, 265) in 254 runs
255
update (164, 267) in 255 runs
256
update (165, 266) in 256 runs
257
update (165, 265) in 257 runs
258
update (165, 267) in 258 runs
259
update (166, 266) in 259 runs
260
update (166, 265) in 260 runs
261
update (166, 267) in 261 runs
262
update (167, 266) in 262 runs
263
update (167, 265) in 263 runs
264
update (167, 267) in 264 runs
265
update (168, 266) in 265 runs
266
update (168, 265) in 266 runs
267
update (168, 267) in 267 runs
268
update (169, 266) in 268 runs
269
update (169, 265) in 269 runs
270
update (169, 267) in 270 runs
271
update (170, 266) in 271 runs
272
update (170, 265) in 272 runs
273
update (170, 267) in 273 runs
274
update (171, 266) in 274 runs
275
update (171, 265) in 275 runs
276
update (171, 267) in 276 runs
277
update (172, 266) in 277 runs
278
update (172, 265) in 278 runs
279
update (172, 267) in 279 runs
280
update (173, 266) in 280 runs
281
update (173, 265) in 281 runs
282
update (173, 267) in 282 runs
283
update (17

500
update (246, 265) in 500 runs
501
update (246, 267) in 501 runs
502
update (247, 266) in 502 runs
503
update (247, 265) in 503 runs
504
update (247, 267) in 504 runs
505
update (248, 266) in 505 runs
506
update (248, 265) in 506 runs
507
update (248, 267) in 507 runs
508
update (249, 266) in 508 runs
509
update (249, 265) in 509 runs
510
update (249, 267) in 510 runs
511
update (250, 266) in 511 runs
512
update (250, 265) in 512 runs
513
update (250, 267) in 513 runs
514
update (251, 266) in 514 runs
515
update (251, 265) in 515 runs
516
update (251, 267) in 516 runs
517
update (252, 266) in 517 runs
518
update (252, 265) in 518 runs
519
update (252, 267) in 519 runs
520
update (253, 266) in 520 runs
521
update (253, 265) in 521 runs
522
update (253, 267) in 522 runs
523
update (254, 266) in 523 runs
524
update (254, 265) in 524 runs
525
update (254, 267) in 525 runs
526
update (255, 266) in 526 runs
527
update (255, 265) in 527 runs
528
update (255, 267) in 528 runs
529
update (25

743
update (327, 265) in 743 runs
744
update (327, 267) in 744 runs
745
update (328, 266) in 745 runs
746
update (328, 265) in 746 runs
747
update (328, 267) in 747 runs
748
update (329, 266) in 748 runs
749
update (329, 265) in 749 runs
750
update (329, 267) in 750 runs
751
update (330, 266) in 751 runs
752
update (330, 265) in 752 runs
753
update (330, 267) in 753 runs
754
update (331, 266) in 754 runs
755
update (331, 265) in 755 runs
756
update (331, 267) in 756 runs
757
update (332, 266) in 757 runs
758
update (332, 265) in 758 runs
759
update (332, 267) in 759 runs
760
update (333, 266) in 760 runs
761
update (333, 265) in 761 runs
762
update (333, 267) in 762 runs
763
update (334, 266) in 763 runs
764
update (334, 265) in 764 runs
765
update (334, 267) in 765 runs
766
update (335, 266) in 766 runs
767
update (335, 265) in 767 runs
768
update (335, 267) in 768 runs
769
update (336, 266) in 769 runs
770
update (336, 265) in 770 runs
771
update (336, 267) in 771 runs
772
update (33

986
update (408, 265) in 986 runs
987
update (408, 267) in 987 runs
988
update (409, 266) in 988 runs
989
update (409, 265) in 989 runs
990
update (409, 267) in 990 runs
991
update (410, 266) in 991 runs
992
update (410, 265) in 992 runs
993
update (410, 267) in 993 runs
994
update (411, 266) in 994 runs
995
update (411, 265) in 995 runs
996
update (411, 267) in 996 runs
997
update (412, 266) in 997 runs
998
update (412, 265) in 998 runs
999
update (412, 267) in 999 runs
1000
update (413, 266) in 1000 runs
1001
update (413, 265) in 1001 runs
1002
update (413, 267) in 1002 runs
1003
update (414, 266) in 1003 runs
1004
update (414, 265) in 1004 runs
1005
update (414, 267) in 1005 runs
1006
update (415, 266) in 1006 runs
1007
update (415, 265) in 1007 runs
1008
update (415, 267) in 1008 runs
1009
update (416, 266) in 1009 runs
1010
update (416, 265) in 1010 runs
1011
update (416, 267) in 1011 runs
1012
update (417, 266) in 1012 runs
1013
update (417, 265) in 1013 runs
1014
update (417, 26

In [79]:
print p_result[(423, 266)].gi_d
print p_result[(423, 266)].gi_v


[   0  686  900  960 1080]
[   inf     0.  1440.     0.]


In [87]:
ti = p_result[(423, 266)].gi_d[np.argmin(p_result[(423, 266)].gi_v)]
result_path = path_selection(p_result, ti, (142, 328), (423, 266), df, [], [], False)

/home/k1758472/.conda/envs/DS_Py27/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [88]:
result_path

{'arrival_time': [686,
  684,
  682,
  680,
  678,
  676,
  674,
  672,
  670,
  668,
  666,
  664,
  662,
  660,
  658,
  656,
  654,
  652,
  650,
  648,
  646,
  644,
  642,
  640,
  638,
  636,
  634,
  632,
  630,
  628,
  626,
  624,
  622,
  620,
  618,
  616,
  614,
  612,
  610,
  608,
  606,
  604,
  602,
  600,
  598,
  596,
  594,
  592,
  590,
  588,
  586,
  584,
  582,
  580,
  578,
  576,
  574,
  572,
  570,
  568,
  566,
  564,
  562,
  560,
  558,
  556,
  554,
  552,
  550,
  548,
  546,
  544,
  542,
  540,
  538,
  536,
  534,
  532,
  530,
  528,
  526,
  524,
  522,
  520,
  518,
  516,
  514,
  512,
  510,
  508,
  506,
  504,
  502,
  500,
  498,
  496,
  494,
  492,
  490,
  488,
  486,
  484,
  482,
  480,
  478,
  476,
  474,
  472,
  470,
  468,
  466,
  464,
  462,
  460,
  458,
  456,
  454,
  452,
  450,
  448,
  446,
  444,
  442,
  440,
  438,
  436,
  434,
  432,
  430,
  428,
  426,
  424,
  422,
  420,
  418,
  416,
  414,
  412,
  410,
  408,
  40

In [89]:
def translate_num_to_time(x):
    hour = x // 60
    minutes = x % 60
    
    if hour < 10:
        hour = '0' + str(hour)
    else:
        hour = str(hour)
        
    if minutes < 10:
        minutes = '0' + str(minutes)
    else:
        minutes = str(minutes)
        
    return hour + ':' + minutes
    
    

In [90]:
pd.DataFrame({'Destination City ID': 1, 'Date ID': 1, 'time': list(map(translate_num_to_time, result_path['arrival_time'])), 'x-axis': [i for i, j in result_path['path']], 'y-axis': [j for i, j in result_path['path']]})

,Date ID,Destination City ID,time,x-axis,y-axis
0,1,1,11:26,423,266
1,1,1,11:24,422,266
2,1,1,11:22,421,266
3,1,1,11:20,420,266
4,1,1,11:18,419,266
5,1,1,11:16,418,266
6,1,1,11:14,417,266
7,1,1,11:12,416,266
8,1,1,11:10,415,266
9,1,1,11:08,414,266
